<a href="https://www.kaggle.com/code/arghya123r/binary-classification-machine-failure?scriptVersionId=139687118" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s3e17/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s3e17/test.csv")

In [4]:
df_train.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [5]:
df_train.columns = [col.replace('[', '').replace(']', '').replace('<', '') if col not in ['Type'] else col for col in df_train.columns]
df_test.columns = [col.replace('[', '').replace(']', '').replace('<', '') if col not in ['Type'] else col for col in df_test.columns]
df_train.columns

Index(['id', 'Product ID', 'Type', 'Air temperature K',
       'Process temperature K', 'Rotational speed rpm', 'Torque Nm',
       'Tool wear min', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

In [6]:
X = df_train.copy()
X_test = df_test.copy()

In [7]:
X.head()

,id,Product ID,Type,Air temperature K,Process temperature K,Rotational speed rpm,Torque Nm,Tool wear min,Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [8]:
X_test.head()

,id,Product ID,Type,Air temperature K,Process temperature K,Rotational speed rpm,Torque Nm,Tool wear min,TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0


In [9]:
X= X.drop(['id'], axis = 1)
X_test = X_test.drop(['id'], axis = 1)

Dropping the Id column as it has no effect on the dataset. Same goes for the 'Product ID'.

In [10]:
X = X.drop(['Product ID'], axis = 1)
X_test = X_test.drop(['Product ID'], axis = 1)

In [11]:
X.head()

,Type,Air temperature K,Process temperature K,Rotational speed rpm,Torque Nm,Tool wear min,Machine failure,TWF,HDF,PWF,OSF,RNF
0,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [12]:
X_test.head()

,Type,Air temperature K,Process temperature K,Rotational speed rpm,Torque Nm,Tool wear min,TWF,HDF,PWF,OSF,RNF
0,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,M,303.4,312.3,1515,41.3,114,0,0,0,0,0


In [13]:
def score_dataset(X_train, X_valid, y_train, y_valid):
        model = RandomForestRegressor(n_estimators = 100, max_depth = 2, random_state = 0)
        model.fit(X_train, y_train)
        preds = model.predict(X_valid)
        return mean_absolute_error(y_valid, preds)

A way to quickly check the loss funcion after making some changes to the whole dataset.

In [14]:
y = X.pop('Machine failure')
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Machine failure, dtype: int64

In [15]:
X['TrqetoSpdRatio'] = abs(X['Torque Nm']/X['Rotational speed rpm'])
X['TempRation'] = abs(X['Air temperature K']/X['Process temperature K'])
X['TempDiff'] = (X['Process temperature K']-X['Air temperature K'])
X['Tool_Prcs'] = X['Tool wear min'] * X['Process temperature K']

Making Some new columns of Data with almost no domain knowledge.

In [16]:
X_test['TrqetoSpdRatio'] = abs(X_test['Torque Nm']/X_test['Rotational speed rpm'])
X_test['TempRation'] = abs(X_test['Air temperature K']/X_test['Process temperature K'])
X_test['TempDiff'] = (X_test['Process temperature K']-X_test['Air temperature K'])
X_test['Tool_Prcs'] = X_test['Tool wear min'] * X_test['Process temperature K']

In [17]:
X = X.replace({"Type":{"L":1,"M":0, "H":-1}})
X_test = X_test.replace({"Type":{"L":1,"M":0,"H":-1}})

Replacing the types column with numerical values.

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
X_train.head()

,Type,Air temperature K,Process temperature K,Rotational speed rpm,Torque Nm,Tool wear min,TWF,HDF,PWF,OSF,RNF,TrqetoSpdRatio,TempRation,TempDiff,Tool_Prcs
131389,1,302.5,311.1,1389,53.5,135,0,0,0,0,0,0.038517,0.972356,8.6,41998.5
50495,1,301.0,312.0,1675,30.7,113,0,0,0,0,0,0.018328,0.964744,11.0,35256.0
37400,1,300.7,310.5,1541,36.0,194,0,0,0,0,0,0.023361,0.968438,9.8,60237.0
89228,-1,303.3,312.6,1611,35.0,125,0,0,0,0,0,0.021726,0.970250,9.3,39075.0
36393,0,298.6,309.5,1426,45.4,36,0,0,0,0,0,0.031837,0.964782,10.9,11142.0


In [19]:
score_dataset(X_train,X_valid,y_train,y_valid)

0.013812109915263689

All the values here in the parameters are Random.

In [20]:
model = XGBRegressor(n_estimators= 800, learning_rate=0.03, n_jobs=4, max_depth=5, early_stopping_rounds=5)
model.fit(X_train,y_train,
         eval_set=[(X_valid, y_valid)],
         verbose=False)
preds=model.predict(X_valid)

In [21]:
mean_absolute_error(y_valid,preds)

0.008141854604129082

In [22]:
X_test.head()

,Type,Air temperature K,Process temperature K,Rotational speed rpm,Torque Nm,Tool wear min,TWF,HDF,PWF,OSF,RNF,TrqetoSpdRatio,TempRation,TempDiff,Tool_Prcs
0,1,302.3,311.5,1499,38.0,60,0,0,0,0,0,0.025350,0.970465,9.2,18690.0
1,1,301.7,311.0,1713,28.8,17,0,0,0,0,0,0.016813,0.970096,9.3,5287.0
2,1,301.3,310.4,1525,37.7,96,0,0,0,0,0,0.024721,0.970683,9.1,29798.4
3,0,300.1,309.6,1479,47.6,5,0,0,0,0,0,0.032184,0.969315,9.5,1548.0
4,0,303.4,312.3,1515,41.3,114,0,0,0,0,0,0.027261,0.971502,8.9,35602.2


In [23]:
Final_pred=model.predict(X_test)
print(Final_pred)

[0.00222076 0.00215632 0.00215632 ... 0.00215632 0.00215632 0.00222076]


In [24]:
sub = pd.read_csv('/kaggle/input/playground-series-s3e17/sample_submission.csv')
sub.head()

,id,Machine failure
0,136429,0.5
1,136430,0.5
2,136431,0.5
3,136432,0.5
4,136433,0.5


In [25]:
sub['Machine failure'] = Final_pred
sub

,id,Machine failure
0,136429,0.002221
1,136430,0.002156
2,136431,0.002156
3,136432,0.002681
4,136433,0.002587
...,...,...
90949,227378,0.002221
90950,227379,0.002156
90951,227380,0.002156
90952,227381,0.002156


In [26]:
sub.to_csv('submission.csv', index = False )